In [36]:
%pip install gmpy2 cryptography
import os
import time
import itertools
import gmpy2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\HUAWEI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [37]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [38]:
'''
     код с использованием AES-GCM

    Не шифруйте файл размером более 64 ГБ, поскольку у GCM есть ограничения!!!
'''

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os, sys, struct

READ_SIZE = 4096

def encrypt_file(plainpath, cipherpath, password):
    # Derive key with a random 16-byte salt
    salt = os.urandom(16)
    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1,
                    backend=default_backend())
    key = kdf.derive(password)

    # Generate a random 96-bit IV.
    iv = os.urandom(12)

    # Construct an AES-GCM Cipher object with the given key and IV.
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
    backend=default_backend()).encryptor()

    associated_data = iv + salt

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)
    with open(cipherpath, "wb+") as fcipher:
        # Make space for the header (12 + 16 + 16), overwritten last
        fcipher.write(b"\x00"*(12+16+16))

        # Encrypt and write the main body
        with open(plainpath, "rb") as fplain:
            for plaintext in iter(lambda: fplain.read(READ_SIZE), b''):
                ciphertext = encryptor.update(plaintext)
                fcipher.write(ciphertext)
            ciphertext = encryptor.finalize() # Always b''.
            fcipher.write(ciphertext) # For clarity

            header = associated_data + encryptor.tag
            fcipher.seek(0,0)
            fcipher.write(header)

def decrypt_file(cipherpath, plainpath, password):
    with open(cipherpath, "rb") as fcipher:
    # read the IV (12 bytes) and the salt (16 bytes)
        associated_data = fcipher.read(12+16)

        iv = associated_data[0:12]
        salt = associated_data[12:28]

    # derive the same key from the password + salt
        kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1, backend=default_backend())
        key = kdf.derive(password)

        # get the tag. GCM tags are always 16 bytes
        tag = fcipher.read(16)

        # Construct an AES-GCM Cipher object with the given key and IV
        # For decryption, the tag is passed in as a parameter
        decryptor = Cipher(
                algorithms.AES(key),
                modes.GCM(iv, tag),
                backend=default_backend()).decryptor()
        decryptor.authenticate_additional_data(associated_data)

        with open(plainpath, "wb+") as fplain:
            for ciphertext in iter(lambda: fcipher.read(READ_SIZE),b''):
                plaintext = decryptor.update(ciphertext)
                fplain.write(plaintext)

In [39]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

key = AESGCM.generate_key(bit_length=256)
aesgcm = AESGCM(key)
nonce = os.urandom(12)  # Recommended size for GCM nonce

plaintext = b"Dinazavrik"
associated_data = b"Additional data"

# Encrypt
ciphertext = aesgcm.encrypt(nonce, plaintext, associated_data)
print(f"Ciphertext: {ciphertext}")

# Decrypt
decrypted_text = aesgcm.decrypt(nonce, ciphertext, associated_data)
print(f"Decrypted Message: {decrypted_text.decode()}")

Ciphertext: b'\x1f\xc7\xe4w\xc7te\x92\x9e\xf6\x1b\x1bN\xeb\x950\xb1\x08\x1c\xbfX\xc8\xaaG1J'
Decrypted Message: Dinazavrik


In [40]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

# Generate RSA keys
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
public_key = private_key.public_key()

# Encrypting
message = b"I love my cat"
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
)
print(f"Ciphertext: {ciphertext}")

# Decrypting
plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
)
print(f"Decrypted Message: {plaintext.decode()}")

Ciphertext: b'\x97\xc9D\x90\xda\\\x97\xdc\xd7\x83U\xffq\xc5\xf66\x8d2A\xfe\r\xe4\xf0\x0c\x89\x05\xc0\xd0\xc1o!\xd5\x82 B\x00\xf4\xfe\x00\xfb\xce\x8e\xba\xbew%\xf3\xa1\t\x18\x1eH1M\xb5\xe03\xff\xceG\x8fE\x04\xcc\x12\x15F\xddmy\xf0\xb1A\xc7\x01`\xd1\xae\xcb\x10^`\xf7\r\xdf\xbb\x05Q\x19\xa4\xc8G\xae\x95\x1eOwX=\x97\x9e\x7f\xf6P\xe9\xb5]\x02\x9f!h\xc0\x8d\xb9\xa8\x89k\xc6Mx\x15\x9c?\x06\xf2\xb8\x0b\x8e\xee\xbf\x10\xa1]\x1fG\x95\x91\xbb\xe0\xa1gE\x8b\x18c\x8a#y\x0c8Z\x04[\x01\x91\x1eGA;t\xc8\xf1FNe\xc5\xeb\xa5\xe8\xa8\no/C9\xf3]hq\x15L\x19\xa6\xa0\x84Z%\xbc59\xbe\x87!\xc5\xc9\xa2\xf0tt\x9f\xbfyt\xa0\x8e\x9b\xa6\xfa\x9cl\x9dN\xf0\x07\x96\xc8\x07T\xaf\xe2\xe2\xc8\xca\x85\x9b\x9bm\xf9\x1c\xf9k!\xc2\xb2 \xba\xa6\xf3_d\x93\xa2\x8c\xd6\x95P+l-\xa5\xbf\xe6\xc5\x12\xc7d'
Decrypted Message: I love my cat


In [41]:
from cryptography.hazmat.primitives import serialization

# Sign data
signature = private_key.sign(
    message,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256()
)
print(f"Signature: {signature}")

# Verify signature
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    print("Signature Verified")
except Exception as e:
    print(f"Signature Verification Failed: {e}")

Signature: b'\x7f\xde\xfc\xf8\x7fU{Sr\x16K\n\xa4\xf1\xb5\xdb\xc1E\x15\x98i\xb3\xb4"\xeb\xbf\xd8\x166{\x84\xf7\x98y\xc1\xbeS\x95H\xab!\xab N\xa4\xcf$]y.;\x94N\x17w3\xc70<f\x0c\x92\xf2\xdbP:I%HO\xc5\x1d*+3\xdc\xc3\x95M\x9c?\xb3R\xb3\x82\x99\xeay\xbe\xe0\x1e\x82a\x94\xb0|\x89\x8a\xe9s\xa9\xf1r\xd8\x99\xcbY\x9fI\xfd\xd8\xd40\x0c\x05\x01f>\xe8\xfa\xd4\xa5\x95\x8e?\x02\xe4\xeem\xd0\x18\xbf\x968\n\x7f\x8c\xe2]\xcco\xee\xe5\x12"\xc0\x03\xe6\xdawZ9\xf3\x9c\xb2-\xc92\xec\x88\xfa2\xfd\xc0V\x1e\xfa\x81\xfe\x04K\x8ez*\xf1\xe6C\xf6\xbd\x0b\xbeU\xd2\xd34\x81\x9d\xb8\xb1\x80\x07\r\x1c\x01t\x8c\xdc\x96:\x08\x8f\xb3 \x17\'Z\x10\xafa\x8c\x85\xf5\x89\xa1\x18\xa8\xe2R\x12\xc8\x16Vm\xe6\xf4\xbaX\xfc\x9d:\x15k\x10b\x86\xe4%$nA)\n\xdcaQ\xa9[\x95\x04\xae\x1c=\xd2\x9d\xd0\x8d'
Signature Verified


In [42]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.backends import default_backend
import os
from cryptography.hazmat.primitives import hashes

digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"Message to hash")
hash_result = digest.finalize()
print(f"SHA-256 Hash: {hash_result.hex()}")

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=os.urandom(16),
    iterations=100000,
    backend=default_backend()
)
key = kdf.derive(b"my password")
print(f"Derived Key: {key.hex()}")

kdf = Scrypt(
    salt=os.urandom(16),
    length=32,
    n=2**14,
    r=8,
    p=1,
    backend=default_backend()
)
key = kdf.derive(b"my password")
print(f"Derived Key: {key.hex()}")

SHA-256 Hash: f1aa45b0f5f6703468f9b9bc2b9874d4fa6b001a170d0f132aa5a26d00d0c7e5
Derived Key: 14011f65f6a4e33d6585c47033dee03297754a3f6458208af5d08c80d1f5bfed
Derived Key: d80d581a1500a4d1813403f9fb06b01637e769c3cd6fd4f3a9594115bd7f3dc1


In [43]:
import secrets

# Сгенерируйте случайную соль в виде шестнадцатеричной строки
salt = secrets.token_hex(16)  # 16 байт случайности — распространённый выбор
print("Соль:", salt)

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = AESGCM.generate_key(bit_length=128)
aesgcm = AESGCM(key)
nonce = os.urandom(12)
ct = aesgcm.encrypt(nonce, data, aad)
aesgcm.decrypt(nonce, ct, aad)

Соль: fad24cd442ce3953c2105a0018e90bfd


b'a secret message'

In [44]:
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = AESGCM.generate_key(bit_length=128)
aesgcm = AESGCM(key)
nonce = os.urandom(12)
ct = aesgcm.encrypt(nonce, data, aad)
aesgcm.decrypt(nonce, ct, aad)


b'a secret message'

In [35]:
import os
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = ChaCha20Poly1305.generate_key()
chacha = ChaCha20Poly1305(key)
nonce = os.urandom(12)
ct = chacha.encrypt(nonce, data, aad)
chacha.decrypt(nonce, ct, aad)

b'a secret message'